In [8]:
import numpy as np
# Surgery type attributes (previously defined)
surgery_type_attributes = {
    "Cardiovascular": {"duration": 4, "variance": 1.0, "priority": 5},
    "Neurological": {"duration": 2, "variance": 1.2, "priority": 4},
    "Orthopedic": {"duration": 3, "variance": 0.5, "priority": 3},
    "Gastrointestinal": {"duration": 4, "variance": 0.7, "priority": 2},
    "Cosmetic": {"duration": 5, "variance": 0.3, "priority": 1},
}

total_surgeries_of_each_type = {1: 9000, 2: 1, 3: 1, 4: 1, 5: 1}

type_names = {
    1: "Cardiovascular",
    2: "Neurological",
    3: "Orthopedic",
    4: "Gastrointestinal",
    5: "Cosmetic",
}

# Generate the surgeries list based on the new structure
surgeries = [
    f"{type_names[type_id]} Surgery {i}"
    for type_id, count in total_surgeries_of_each_type.items()
    for i in range(1, count + 1)
]

# Extend surgeries list to include an intake_date
extended_surgeries = [
    {
        "id": f"{type_names[type_id]}_Surgery_{i}_Intake_{np.random.randint(1, 11)}",  # Unique identifier
        "name": f"{type_names[type_id]} Surgery {i}",
        "type_id": type_id,
        "intake_date": np.random.randint(1, 11)  # Random intake date between 1 and 10
    }
    for type_id, count in total_surgeries_of_each_type.items()
    for i in range(1, count + 1)
]


In [9]:
extended_surgeries

[{'id': 'Cardiovascular_Surgery_1_Intake_4',
  'name': 'Cardiovascular Surgery 1',
  'type_id': 1,
  'intake_date': 4},
 {'id': 'Cardiovascular_Surgery_2_Intake_1',
  'name': 'Cardiovascular Surgery 2',
  'type_id': 1,
  'intake_date': 8},
 {'id': 'Cardiovascular_Surgery_3_Intake_3',
  'name': 'Cardiovascular Surgery 3',
  'type_id': 1,
  'intake_date': 5},
 {'id': 'Cardiovascular_Surgery_4_Intake_4',
  'name': 'Cardiovascular Surgery 4',
  'type_id': 1,
  'intake_date': 10},
 {'id': 'Cardiovascular_Surgery_5_Intake_1',
  'name': 'Cardiovascular Surgery 5',
  'type_id': 1,
  'intake_date': 6},
 {'id': 'Cardiovascular_Surgery_6_Intake_6',
  'name': 'Cardiovascular Surgery 6',
  'type_id': 1,
  'intake_date': 6},
 {'id': 'Cardiovascular_Surgery_7_Intake_4',
  'name': 'Cardiovascular Surgery 7',
  'type_id': 1,
  'intake_date': 3},
 {'id': 'Cardiovascular_Surgery_8_Intake_8',
  'name': 'Cardiovascular Surgery 8',
  'type_id': 1,
  'intake_date': 2},
 {'id': 'Cardiovascular_Surgery_9_Intak

In [10]:
from gurobipy import Model, GRB
import numpy as np

def schedule_surgeries_rolling(surgeries, surgery_type_attributes, day_capacities, beta=0.5):
    """
    Schedule surgeries within a rolling horizon optimization framework, assuming one room per day.
    
    Parameters:
    - surgeries: List of dictionaries representing surgeries with 'id', 'type_id', and 'intake_date'.
    - surgery_type_attributes: Dictionary mapping surgery types to their attributes (duration, variance, priority).
    - day_capacities: Dictionary with days as keys and available hours as values for the single room available that day.
    - beta: Multiplier for variance-based slack time adjustment.
    
    Returns:
    - scheduled_surgeries: Dictionary with scheduled day for each surgery. Room is omitted since there's only one per day.
    - updated_day_capacities: Updated day capacities after scheduling surgeries.
    """
    
    m = Model("SurgeryScheduling")

    # Define decision variables for surgeries scheduled on each day
    surgery_scheduled = m.addVars(
        [(surgery['id'], d) for surgery in surgeries for d in day_capacities.keys()], 
        vtype=GRB.BINARY, 
        name="schedule"
    )

    # Ensure that surgery_type_attributes includes entries for all possible type_id values
    type_ids = set(surgery['type_id'] for surgery in surgeries)
    for type_id in type_ids:
        if type_id not in surgery_type_attributes:
            surgery_type_attributes[type_id] = {
                'duration': 1,  # Placeholder value for duration
                'variance': 0,  # Placeholder value for variance
                'priority': 1   # Placeholder value for priority
            }

    # Define the objective function
    m.setObjective(
        sum(
            surgery_scheduled[surgery['id'], d] * 
            surgery_type_attributes[surgery['type_id']]['priority'] * 
            beta*(max(day_capacities.keys()) - d + surgery['intake_date'])  # Example weight calculation, assumes days are numeric
            for surgery in surgeries for d in day_capacities.keys()
        ),
        GRB.MAXIMIZE
    )

    # Add constraints
    # Each surgery can only be scheduled once
    for surgery in surgeries:
        m.addConstr(
            sum(surgery_scheduled[surgery['id'], d] for d in day_capacities.keys()) <= 1,
            f"OneTime_{surgery['id']}"
        )

    # Respect the available hours for the single room each day
    for d in day_capacities.keys():
        m.addConstr(
            sum(surgery_scheduled[surgery['id'], d] * surgery_type_attributes[surgery['type_id']]['duration'] for surgery in surgeries) <= day_capacities[d],
            f"RoomAvailability_{d}"
        )

    # Optimize the model
    m.optimize()

    # Extract the scheduling results
    scheduled_surgeries = {
        surgery['id']: {"day": d}
        for surgery in surgeries for d in day_capacities.keys()
        if surgery_scheduled[surgery['id'], d].X > 0.5
    }

    # Update day capacities based on scheduled surgeries
    updated_day_capacities = day_capacities.copy()
    for surgery_id, info in scheduled_surgeries.items():
        updated_day_capacities[info['day']] -= surgery_type_attributes[surgeries[0]['type_id']]['duration']  # Assuming all surgeries have the same duration
    
    return scheduled_surgeries, updated_day_capacities


# Example usage:

# Set the capacity for each day to 24 hours
day_capacities = {day: 24 for day in range(1, num_days + 1)}

# Now, call the function with the extended surgeries list and other parameters
scheduled_surgeries, updated_day_capacities = schedule_surgeries_rolling(
    extended_surgeries, 
    surgery_type_attributes,
    day_capacities
)

# Print the scheduled surgeries for each day
for surgery_id, info in scheduled_surgeries.items():
    print(f"Surgery {surgery_id} scheduled on day {info['day']}")

# Print the updated day capacities
print("Updated day capacities:")
for day, capacity in updated_day_capacities.items():
    print(f"Day {day}: {capacity} hours")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
from gurobipy import Model, GRB
import numpy as np

hours_per_day = 24  # Assume 3 rooms of 8hrs each
beta = 0.0  # Initial beta value for slack time adjustment

# Define the number of days in your planning horizon
num_days = 30

# Set the capacity for each day to 24 hours
day_capacities = {day: 24 for day in range(1, num_days + 1)}

# Now, call the function with the extended surgeries list and other parameters
scheduled_surgeries = schedule_surgeries_rolling(
    extended_surgeries, 
    surgery_type_attributes,
    day_capacities
)

# Since the direct output might be extensive, let's summarize the results
# Counting the number of scheduled surgeries per day and room
scheduled_summary = {}
# Create a list to hold details of scheduled surgeries
scheduled_surgeries_list = []

# Set the capacity for each day to 24 hours
day_capacities = {day: 24 for day in range(1, num_days + 1)}

# Now, call the function with the extended surgeries list and other parameters
scheduled_surgeries, updated_day_capacities = schedule_surgeries_rolling(
    extended_surgeries, 
    surgery_type_attributes,
    day_capacities
)

# Print the scheduled surgeries for each day
for surgery_id, info in scheduled_surgeries.items():
    print(f"Surgery {surgery_id} scheduled on day {info['day']}")

# Print the updated day capacities
print("Updated day capacities:")
for day, capacity in updated_day_capacities.items():
    print(f"Day {day}: {capacity} hours")



Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 35 rows, 150 columns and 300 nonzeros
Model fingerprint: 0xe930280e
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 80.5000000
Presolve removed 35 rows and 150 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 80.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.050000000000e+01, best bound 8.050000000000e+01, gap 0.0000%
Gurobi Optimizer version 11.0.1 b